In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
# Analyzing the true dataset
data_true=pd.read_csv("True.csv")
data_fake=pd.read_csv("Fake.csv")

In [3]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
data_true.tail(10)

,title,text,subject,date
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017"
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017"
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017"
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017"
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017"
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017"


In [6]:
data_fake.tail(10)

,title,text,subject,date
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016"
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016"
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016"
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016"
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016"
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016"


In [7]:
data_true["class"]=0

In [8]:
data_fake["class"]=1

In [9]:
data_true.shape, data_fake.shape

((21417, 5), (23481, 5))

In [10]:
#Getting only the last 10 data for further processing
data_true_manual_testing=data_true.tail(10)
for i in range(21416, 21406, -1):
    data_true.drop([i], axis=0, inplace=True)
data_fake_manual_testing=data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop([i], axis=0, inplace=True)

In [11]:
data_true_manual_testing["class"]=0
data_fake_manual_testing["class"]=1

C:\Users\SANJANA\AppData\Local\Temp\ipykernel_8356\2927159566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing["class"]=0
C:\Users\SANJANA\AppData\Local\Temp\ipykernel_8356\2927159566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing["class"]=1


In [12]:
data_true_manual_testing.shape, data_fake_manual_testing.shape

((10, 5), (10, 5))

In [13]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",0
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",0
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",0
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",0
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",0
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",0


In [14]:
##Merging both data to get a clear data
df = pd.concat([data_fake, data_true], axis=0)
df.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [15]:
df.shape

(44878, 5)

In [16]:
exclude_cols = ['title', 'subject', 'date']
df = df.drop(columns=exclude_cols)

In [17]:
df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [18]:
df.shape

(44878, 2)

In [19]:
df.isna().sum()

text     0
class    0
dtype: int64

In [20]:
df.nunique()

text     38637
class        2
dtype: int64

***Our goal here is to develop a model which is capable of detecting the fake and the true news so for that
we are currently not focusing on the EDA part like dropping uniques instead, we have already took the 
important columns and we are working on that alone***

In [22]:
df["class"].value_counts()*100/len(df)

class
1    52.299568
0    47.700432
Name: count, dtype: float64

In [23]:
def wordopt(text):
    text=text.lower()
    text=re.sub('\[.*?\]'," ",text)
    text=re.sub("\\W"," ",text)
    text=re.sub('https?://\S+|www.\.S+'," ",text)
    text=re.sub('<.*?>+'," ",text)
    text=re.sub('[%s]'% re.escape(string.punctuation)," ",text)
    text=re.sub('\n'," ",text)
    text=re.sub('\w*\d\w*'," ",text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\SANJANA\AppData\Local\Temp\ipykernel_8356\296948800.py:3: SyntaxWarning: invalid escape sequence '\['
  text=re.sub('\[.*?\]'," ",text)
C:\Users\SANJANA\AppData\Local\Temp\ipykernel_8356\296948800.py:5: SyntaxWarning: invalid escape sequence '\S'
  text=re.sub('https?://\S+|www.\.S+'," ",text)
C:\Users\SANJANA\AppData\Local\Temp\ipykernel_8356\296948800.py:9: SyntaxWarning: invalid escape sequence '\w'
  text=re.sub('\w*\d\w*'," ",text)


In [24]:
df["text"]=df["text"].apply(wordopt)

In [25]:
df["text"]

0        donald trump just couldn t wish all americans ...
1        house intelligence committee chairman devin nu...
2        on friday  it was revealed that former milwauk...
3        on christmas day  donald trump announced that ...
4        pope francis used his annual christmas day mes...
                               ...                        
21402    on board a u s  military aircraft  reuters    ...
21403    washington  reuters    the united states sugge...
21404    washington  reuters    the united states has d...
21405    islamabad  reuters    outlining a new strategy...
21406    geneva  reuters    north korea and the united ...
Name: text, Length: 44878, dtype: object

In [26]:
#Splitting the data
x=df["text"]
y=df["class"]

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=10)

In [28]:
#Using vectorizer for further encoding process and to similarity check
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(x_train)
xv_test=vectorizer.transform(x_test)

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(xv_train, y_train)   # training
lr_pred = lr.predict(xv_test)   # prediction

In [30]:
lr.score(xv_test, y_test)

0.9846256684491979

In [31]:
print("Classification report using LogisticRegression: \n",classification_report(y_test, lr_pred))

Classification report using LogisticRegression: 
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      2182
           1       0.99      0.98      0.98      2306

    accuracy                           0.98      4488
   macro avg       0.98      0.98      0.98      4488
weighted avg       0.98      0.98      0.98      4488



In [32]:
from sklearn.tree import DecisionTreeClassifier
Dt = DecisionTreeClassifier()
Dt.fit(xv_train, y_train)

DecisionTreeClassifier()

In [33]:
Dt_pred = Dt.predict(xv_test)

In [34]:
Dt.score(xv_test, y_test)

0.9939839572192514

In [35]:
print("Classification report using DecisionTreeClassifier: \n",classification_report(y_test, lr_pred))

Classification report using DecisionTreeClassifier: 
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      2182
           1       0.99      0.98      0.98      2306

    accuracy                           0.98      4488
   macro avg       0.98      0.98      0.98      4488
weighted avg       0.98      0.98      0.98      4488



In [36]:
from sklearn.ensemble import GradientBoostingClassifier
Gr = GradientBoostingClassifier()
Gr.fit(xv_train, y_train)

GradientBoostingClassifier()

In [38]:
Gr_pred = Gr.predict(xv_test)

In [40]:
print("Classification report using GradientBoostingClassifier: \n",classification_report(y_test, Gr_pred))

Classification report using GradientBoostingClassifier: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      2182
           1       1.00      0.99      0.99      2306

    accuracy                           0.99      4488
   macro avg       0.99      0.99      0.99      4488
weighted avg       0.99      0.99      0.99      4488



In [42]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(xv_train, y_train)

RandomForestClassifier()

In [43]:
rf_pred = rf.predict(xv_test)

In [44]:
print("Classification report using RandomForestClassifier: \n",classification_report(y_test, rf_pred))

Classification report using RandomForestClassifier: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      2182
           1       0.99      0.99      0.99      2306

    accuracy                           0.99      4488
   macro avg       0.99      0.99      0.99      4488
weighted avg       0.99      0.99      0.99      4488



## Testing

In [72]:
def output_label(n):
    if n==0:
        return "Not a Fake News"
    elif n==1:
        return "Fake News"
def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    
    new_x_test = new_def_test["text"]
    new_xv_test = vectorizer.transform(new_x_test)
    
    pred_LR = lr.predict(new_xv_test)
    pred_DT = Dt.predict(new_xv_test)
    pred_GB = Gr.predict(new_xv_test)
    pred_RF = rf.predict(new_xv_test)
    
    print("Logistic Regression:", output_label(pred_LR[0]))
    print("Decision Tree:", output_label(pred_DT[0]))
    print("Gradient Boosting:", output_label(pred_GB[0]))
    print("Random Forest:", output_label(pred_RF[0]))


if __name__ == "__main__":
    news = input()
    manual_testing(news)

 Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

Logistic Regression: Fake News
Decision Tree: Fake News
Gradient Boosting: Fake News
Random Forest: Fake News
